In [4]:
import os
import urllib3
import numpy as np 
import pandas as pd


# Goal 

## acquire all pubmed abstracts regarding the query term "hlh" 

1- Using an API call to e-search "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?term=hlh&RetMax=5000" to acquire the IDs of relevant articles

In [1]:
os.getcwd()

'd:\\Zhao\\Coding\\hlh-shawn\\ds'

In [9]:
# Use eSearch API to acquire IDs for texts related to "hlh" 


eSearchURL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?term=hlh&RetMax=5000"

http = urllib3.PoolManager()
eSearchRes = http.request('GET', eSearchURL)

with open("./dataset/hlh_PUBMED_QUERY.xml", 'ab') as f:
    f.write(eSearchRes.data)

In [14]:
# Read from stored XML and parse IDs

import xml.etree.ElementTree as et 

xtree = et.parse("./dataset/hlh_PUBMED_QUERY.xml")
IdNode = xtree.getroot().find("IdList")

ID_LIST = [i.text for i in IdNode.findall("Id")]

In [16]:
# Construct form-data for HTTP POST request to EFetch 

EFETCH_POST_BODY = {
    "db": "pubmed",
    "retmode": "xml",
    "id": ",".join(ID_LIST)
}

In [17]:
# Retrieve XML from EFetch API
eFetchURL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"

http = urllib3.PoolManager()

eFetchRes = http.request(
    "POST",
    eFetchURL,
    fields = EFETCH_POST_BODY,
)

with open("./dataset/hlh_PUBMED_ABSTRACTS.xml", 'ab') as f:
    f.write(eFetchRes.data)

In [20]:
xtree = et.parse("./dataset/hlh_PUBMED_ABSTRACTS.xml")
articleNode = xtree.getroot()
articleNode

<Element 'PubmedArticleSet' at 0x000001D8DC957270>